In [0]:
%pip install -r requirements.txt
%restart_python

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

## Total NZ Household Deposits

The code below cleans the raw input data and converts it into the correct format. It processes the quarterly data by using duplication and interpolation which are joined and saved separately. See datalog for detail on the data

In [0]:
def process_quarterlies_duplicate(df):
    # List to hold monthly duplicated rows
    monthly_rows = []

    for _, row in df.iterrows():
        # Use the quarter-end date from the original row
        quarter_end = pd.to_datetime(row['date'])

        # Generate 3 monthly dates ending at the quarter-end (i.e., previous two months + current)
        months = pd.date_range(end=quarter_end, periods=3, freq='M')

        # Duplicate the row for each month in the quarter
        for m in months:
            new_row = row.copy()
            new_row['date'] = m
            monthly_rows.append(new_row)

    # Create new DataFrame with monthly frequency
    monthly_df = pd.DataFrame(monthly_rows)

    # Format 'date' column to 'YYYY-MM'
    monthly_df['date'] = pd.to_datetime(monthly_df['date']).dt.strftime('%Y-%m')

    return monthly_df


def process_quarterlies_interpolate(df):
    non_date_cols = [x for x in df.columns if x != 'date']
    
    # Create a complete monthly date range
    months = pd.date_range(start=df.date.min(), end=df.date.max(), freq='MS')
    month_df = pd.DataFrame(months, columns=['date'])
    month_df['date'] = month_df['date'].dt.strftime('%Y-%m')

    # Merge with original DataFrame
    month_df = month_df.merge(df, on='date', how='left')

    # Interpolate with direction both ways
    month_df[non_date_cols] = month_df[non_date_cols].interpolate(limit_direction='both')

    return month_df


def clean_and_process_df(df):
    # Standardize column names
    df.columns = [col.lower() for col in df.columns]
    df.columns = (
        df.columns
        .str.replace(r"\s*\(.*?\)", "", regex=True)   # remove text in parentheses
        .str.replace(" ", "_", regex=False)           # replace spaces with underscores
        .str.replace("-", "_", regex=False)           # replace hyphens with underscores
        .str.replace(".", "", regex=False)            # remove dots
        .str.replace("\t", "", regex=False)            # remove tabs
    )

    # Convert 'date' column to datetime and format
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce').dt.strftime('%Y-%m')

    # Drop rows where all elements are NaN
    df = df.dropna(how='all')

    # Convert applicable columns to numeric
    for col in df.columns:
        if col == 'date':
            continue
        if df[col].dtype == object:
            # Remove commas if present
            if df[col].astype(str).str.contains(",", na=False).any():
                df[col] = df[col].str.replace(",", "", regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df


In [0]:
# read raw input
monthly_df = pd.read_csv('../data/raw/total_household_deposits_monthly_data.csv')
quarterly_df = pd.read_csv('../data/raw/total_household_deposits_quarterly_data.csv')

# clean and process df
monthly_df = clean_and_process_df(monthly_df)
quarterly_df = clean_and_process_df(quarterly_df)

# process quarterly data
duplicate_quarterly_df = process_quarterlies_duplicate(quarterly_df) # duplicate quarterly data
interpolated_quarterly_df = process_quarterlies_interpolate(quarterly_df) # interpolate quarterly data

# combine and save
combined_data_duplicated = monthly_df.merge(duplicate_quarterly_df, on = 'date', how = 'left').sort_values(by='date')
combined_data_interpolated = monthly_df.merge(interpolated_quarterly_df, on = 'date', how = 'left').sort_values(by='date')

combined_data_duplicated.to_csv('../data/processed/combined_total_household_data_duplicate.csv', index = False)
combined_data_interpolated.to_csv('../data/processed/combined_total_household_data_interpolate.csv', index = False)

In [0]:
# Sense check if there are gaps or dates missing

def check_missing_dates_plot(df):
    # Drop rows that contain any NaNs
    df_cleaned = df.dropna()

    # List of dates that have no missing values across any column
    complete_dates = list(df_cleaned.date)

    # Create a DataFrame to flag missing dates
    date_check_df = df[['date']].copy()
    date_check_df['missing'] = date_check_df['date'].apply(lambda x: 1 if x in complete_dates else 0)

    # Set date as datetime index for plotting
    date_check_df['date'] = pd.to_datetime(date_check_df['date'])
    date_check_df = date_check_df.set_index('date')

    # Plot the presence of data (1 = complete row, 0 = row with any NaN)
    date_check_df.plot.line(figsize=(15, 5), title='Presence of Complete Data by Date (0 = missing)')

    return date_check_df
  
print('Sense check if any dates are missing')
#check_missing_dates_plot(combined_data_duplicated)
check_missing_dates_plot(combined_data_interpolated)

In [0]:
## write to catalogue
#spark.createDataFrame(combined_data_interpolated).write.mode('overwrite').option('overwriteSchema','true').saveAsTable('deposits_forecasting.default.total_hshld_deposits_input_data')